Initial Imports

In [ ]:
import csv
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd

# Generate map of cities and their respective coordinates

In [ ]:
# Replace 'file.csv' with your CSV file's name and path
file_path = 'data/tr.csv'

# Create an empty dictionary to store city coordinates
city_coordinates = {}

# Open the CSV file and read data
with open(file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        city = row['city']
        lat = float(row['lat'])
        lng = float(row['lng'])
        city_coordinates[city] = (lat, lng)

# Display the dictionary containing city coordinates
for city, coordinates in city_coordinates.items():
    print(f"City: {city}, Coordinates: {coordinates}")
city_coordinates

In [1]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2023-12-21",
	"end_date": "2024-01-04",
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)




Coordinates 52.5483283996582°E 13.407821655273438°N
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                   date  temperature_2m
0   2023-12-21 00:00:00          5.4585
1   2023-12-21 01:00:00          4.8585
2   2023-12-21 02:00:00          6.3585
3   2023-12-21 03:00:00          7.0585
4   2023-12-21 04:00:00          7.2585
..                  ...             ...
355 2024-01-04 19:00:00         -0.2915
356 2024-01-04 20:00:00         -2.1915
357 2024-01-04 21:00:00         -0.9915
358 2024-01-04 22:00:00         -0.1915
359 2024-01-04 23:00:00         -0.6415

[360 rows x 2 columns]
